In [3]:
#import packages
import numpy as np
import random
import pandas as pd
from sympy import *
import statistics
import math

Step 1: Create a dataset with 10,000 rows and 4 random variables: 2 normally distributed, 2 normally distributed.

In [4]:
#create the data

x1 = np.random.normal(loc = 25, scale = 10, size = (10000,)) #generate a vector of 10,000 random variables following a normal dist. centered 25, SD = 10
x2 = np.random.normal(loc = 84, scale = 47, size = (10000,)) #generate vector of 10,000 random variables following normal dist. centered 84, SD = 47
x3 = np.random.uniform(low = 13, high = 92, size = (10000,)) #generate vector of 10,000 random variables following uniform dist. between 13 and 92
x4 = np.random.uniform(low = -23,  high = 15, size = (10000,)) #generate vector of 10,000 random variables following uniform dist. between -23 and 15
x5 = x4 ** 2 #create a vector that is squaring the values of x4




Step 2: add another variable y as a linear combination with coeficcients of your choice, of: the 4 variables above; the square of one of the variables; and some random noise (randomly distruib

In [5]:
y = [] #add empty vector y that will be filled
def create_response_variable (x1,x2,x3,x4,x5): #define a function that will generate our response variable (y)
    b0 = 22                                    # create/pick coefficients to be used in the linear model. These will serve as the 'true' values
    b1 = 3
    b2 = -1.2
    b3 = .78
    b4 = 8.7
    b5 = -14

    noise_lowerbound = -10                    #creating lower and upper bounds for the error/noise that will be added into the linear model
    noise_upperbound = 10

    noise = np.random.uniform(low = noise_lowerbound, high = noise_upperbound)   #creating a varibale 'noise' that generates a random error between the 
                                                                                 # thresholds defined above
    return b0 + b1 * x1 + b2 * x2 + b3 * x3 + b4 * x4 + b5 * x5 + noise          # linear regression equation adding in noise... will generate our response

    
    

In [6]:
for x in range (len(x1)):   #for loop that is generating a response varible for each randomly generated observation and placing the values in the 'y' vector
    current_response = create_response_variable(x1[x], x2[x], x3[x], x4[x], x5[x])      #uses the function defined above
    y.append(current_response)                                                          #creates a y vector with all the response variables

In [8]:
x_data = pd.DataFrame({'x1':x1, 'x2': x2, 'x3':x3, 'x4':x4, 'x5':x5})               #creates a data frame with all the predictor variables
complete_data = pd.DataFrame({'x1':x1, 'x2': x2, 'x3':x3, 'x4':x4, 'x5':x5, 'y':y}) #creates a data framw with all the generated data
complete_data

,x1,x2,x3,x4,x5,y
0,20.981678,77.374484,85.466641,-20.068433,402.741989,-5754.826913
1,12.650763,42.852599,14.946385,14.079190,198.223588,-2631.085254
2,25.593256,73.089906,42.865836,-13.614149,185.345065,-2672.736600
3,18.861400,155.596657,48.533497,-10.824764,117.175521,-1802.740036
4,32.406984,171.150490,51.575323,2.846929,8.105005,-138.297486
...,...,...,...,...,...,...
9995,36.066217,92.960246,23.364174,-6.842300,46.817066,-686.097757
9996,10.577646,113.652121,39.260814,4.557137,20.767496,-306.678144
9997,10.262464,66.366858,41.958422,-5.408287,29.249566,-459.135238
9998,22.478975,31.469969,68.748790,-12.754579,162.679294,-2280.084332


Step 3: Estimate the linear regression coefficients on the whole data using OLS

In [9]:
def OLS(x1,x2,x3,x4,x5,y_vector):                           #define a function that performs OLS regression on our data set containing 5 predictors
    ones = [1 for x in range (len(x1))]                     #generating our column vector of 1s
    X = Matrix([ones, x1, x2, x3, x4, x5]).T                #generating X Matrix
    y = Matrix(y_vector)                                    #generating y Matrix
    coeff_estimates = ((X.T*X).inv())*(X.T)*y               #perform OLS
    return coeff_estimates                                  #return coefficient estimates

In [10]:
population_coefficients = OLS(x1,x2,x3,x4,x5,y)          #run OLS function on our generated data
population_coefficients =np.asarray(population_coefficients) #make the resulting list an array (so we can perform math on it)
print(population_coefficients) #print the array

[[22.1106705192959]
 [2.99735046797379]
 [-1.20127018106406]
 [0.779663155975666]
 [8.69964025771012]
 [-13.9999657946401]]


Step 4: Use Bootstrapping to create 10 other samples from the data you created.

In [11]:
#create 10 bootstraps
bootstrap_number = 10      #define the number of bootstrap samples to be performed
samples = []               #create an empty list of samples

#for loop that is iterating through the number of bootstrap samples specified. It is creating a sample of 1000 observations with replacement from 
#the complete_data data frame. It is appending each bootstrap into a list called "samples".
for i in range(bootstrap_number):       
    bootstrap_sample = complete_data.sample(1000,replace = True)
    samples.append(bootstrap_sample)
    

In [12]:
#create empty lists for the OLS results. We created an empty list for each coefficient as this will be easier and more flexible to work with in 
# future steps vs placing the results in the same list and having to seperate them later.
ols_results = []
b0_results = []
b1_results = []
b2_results = []
b3_results = []
b4_results = []
b5_results = []

#for loop that is iterating through the number of samples. Creating temporary lists of the predictor variables associated with the sampled observations 
# to be used in OLS
for sample in samples:            
    x1_temp = sample['x1'] #creating temporary list of all the x1 predictor variables in the sample being iterated
    x2_temp = sample['x2'] #creating temporary list of all the x2 predictor variables in the sample being iterated
    x3_temp = sample['x3'] #creating temporary list of all the x3 predictor variables in the sample being iterated
    x4_temp = sample['x4'] #creating temporary list of all the x4 predictor variables in the sample being iterated
    x5_temp = sample['x5'] #creating temporary list of all the x5 predictor variables in the sample being iterated
    y_temp = sample['y'] #creating temporary list of all the y response variables in the sample being iterated
    result = OLS(x1_temp, x2_temp, x3_temp, x4_temp, x5_temp, y_temp)   #performing OLS on this current sample
    b0_results.append(result[0])  #appending the results of current sample b0 estimate to the list of all bootstrapped b0 estimates
    b1_results.append(result[1])  #appending the results of current sample b1 estimate to the list of all bootstrapped b0 estimates
    b2_results.append(result[2])  #appending the results of current sample b2 estimate to the list of all bootstrapped b0 estimates
    b3_results.append(result[3])  #appending the results of current sample b3 estimate to the list of all bootstrapped b0 estimates
    b4_results.append(result[4])  #appending the results of current sample b4 estimate to the list of all bootstrapped b0 estimates
    b5_results.append(result[5])  #appending the results of current sample b5 estimate to the list of all bootstrapped b0 estimates

In [13]:
#create a dataframe with all the coefficient estimates from the bootstrap samples. This is to create a nice visual with all the data
bootstrap_coefficients = pd.DataFrame({
    'b0': b0_results,
    'b1': b1_results,
    'b2': b2_results,
    'b3': b3_results,
    'b4': b4_results,
    'b5': b5_results
})

bootstrap_coefficients


,b0,b1,b2,b3,b4,b5
0,23.2160245194421,2.99173908432632,-1.20219928180657,0.765140644388741,8.68794919266544,-14.0008228507664
1,21.3712952125975,3.03415706286296,-1.20281943202738,0.780771164392434,8.74501733529024,-13.9971826470540
2,20.8742998239234,3.01599407759360,-1.19973321542465,0.783949806249421,8.72679741077055,-13.9970597694323
3,21.4000320785097,3.00913941283519,-1.20188261422182,0.790467481530720,8.64930287162081,-14.0017349489948
4,22.5929991120089,2.99703761780106,-1.20713019842660,0.776879957587342,8.68103005844415,-13.9998501836841
5,22.9575366868842,3.00041504071987,-1.20511830401585,0.772549753328631,8.69682971160771,-14.0002880814088
6,22.7156377557157,3.00605899153742,-1.19805559690883,0.761831256753843,8.70451962796693,-14.0022558727366
7,21.6786777031222,3.00396924999642,-1.19769986839025,0.779736027426916,8.71805673872196,-13.9993463352689
8,22.2317307234389,2.99076809072885,-1.20517439574675,0.784656309036311,8.67836463311257,-14.0002934143292
9,21.5954527951776,3.01594570745869,-1.19803546105386,0.772597632701312,8.70622956655894,-14.0002981984096


Step 6: For each coefficient in number 3, compute the standard deviation from the mean of the 10 corresponding coefficients estimated in 5.

In [14]:
#Make each list of coefficients into an array so we can calculate the mean and standard deviation
beta_0_estimates = np.array(b0_results)
beta_1_estimates = np.array(b1_results)
beta_2_estimates = np.array(b2_results)
beta_3_estimates = np.array(b3_results)
beta_4_estimates = np.array(b4_results)
beta_5_estimates = np.array(b5_results)

In [15]:
#calculate the means of the bootstrap coefficients
beta_0_mean = beta_0_estimates.mean()
beta_1_mean = beta_1_estimates.mean()
beta_2_mean = beta_2_estimates.mean()
beta_3_mean = beta_3_estimates.mean()
beta_4_mean = beta_4_estimates.mean()
beta_5_mean = beta_5_estimates.mean()

#place all the means in an object/list called "bootstrap means"
bootstrap_means = [beta_0_mean, beta_1_mean, beta_2_mean, beta_3_mean, beta_4_mean, beta_5_mean] 
bootstrap_means

[22.0633686410820,
 3.00652243358604,
 -1.20178483680226,
 0.776858003339567,
 8.69940971467593,
 -13.9999132302085]

In [16]:
#calculate the standard deviations of the bootstrap
#define a standard deviation function
def standard_deviation(data, mean):
    squared_diff = [(x - mean) ** 2 for x in data]
    mean_squared_diff = sum(squared_diff) / len(data)
    std_dev = math.sqrt(mean_squared_diff)
    return std_dev


In [17]:
#calculate the standard deviation for each of the bootstrapped coefficients
std_b0 = standard_deviation(beta_0_estimates, beta_0_mean)
std_b1 = standard_deviation(beta_1_estimates, beta_1_mean)
std_b2 = standard_deviation(beta_2_estimates, beta_2_mean)
std_b3 = standard_deviation(beta_3_estimates, beta_3_mean)
std_b4 = standard_deviation(beta_4_estimates, beta_4_mean)
std_b5 = standard_deviation(beta_5_estimates, beta_5_mean)

#place the results in an object/list called "bootstrap_std"
bootstrap_std = [std_b0, std_b1, std_b2, std_b3, std_b4, std_b5]
bootstrap_std

[0.7458055595570676,
 0.012448990656339655,
 0.003182560017535219,
 0.008483707095460645,
 0.025845457971438463,
 0.0016121713406450585]

In [20]:
#place mean and std_dev of each bootstrapped coefficient into a data frame for reference.
mean_std_df = pd.DataFrame({
    'Coefficient': ['b0', 'b1', 'b2', 'b3', 'b4', 'b5'],
    'Mean': bootstrap_means,
    'Standard Deviation': bootstrap_std
})

mean_std_df

,Coefficient,Mean,Standard Deviation
0,b0,22.0633686410820,0.745806
1,b1,3.00652243358604,0.012449
2,b2,-1.20178483680226,0.003183
3,b3,0.776858003339567,0.008484
4,b4,8.69940971467593,0.025845
5,b5,-13.9999132302085,0.001612


In [21]:
#define a function that will find how many standard deviations of the bootstrap estimates the true population coefficients (with added error) are away
#from the mean of the bootstrapped coefficient estimates
def std_away_from_mean(population_coefficients, bootstrap_means, bootstrap_std):
    deviations = []
    for pop_coef, bs_mean, bs_std in zip(population_coefficients, bootstrap_means, bootstrap_std):
        deviation = (pop_coef - bs_mean) / bs_std
        deviations.append(deviation)
    return deviations

In [22]:
#run the function above.
std_away = std_away_from_mean(population_coefficients, bootstrap_means, bootstrap_std)

In [23]:
#print results... as we can see.. all are very close to the population estimates.
print(std_away)

[array([0.0634238745041535], dtype=object), array([-0.736763795992912], dtype=object), array([0.161711243577638], dtype=object), array([0.330651754537823], dtype=object), array([0.00892005993631266], dtype=object), array([-0.0326047426115153], dtype=object)]
